# Ejemplo datos aleatorios uniformes y dimensión intrínseca

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 21 de junio de 2025

Se va a probar qué sucede si el dataset se compone de vectores aleatorios uniformes.   
¿los índices podrán acelerar las búsquedas?

1. Se genera un dataset con vectores aleatorios de 128-dimensiones.

2. Se resuelve la búsqueda exacta usando un k-d tree, que resulta ser mucho más lenta que el linear scan.

3. Se resuelven búsquedas aproximadas con k-d tree, que muestra poca ganancia en tiempo.

4. Al graficar el histograma de distancias del dataset, se ve que es simétrico con distancias cercanas al promedio, y al calcular la dimensión intrínseca da un valor grande, cercano a la dimensión real.

5. Finalmente se compara con el histograma de distancias de datos reales, donde se ve un histograma con una forma distinta al anterior. Al calcular la dimensión intrínseca se observa que entrega un valor mucho menor a su dimensión real.


## Generar dataset aleatorio uniforme de 128-d

In [ ]:
import numpy

numpy.random.seed(4)


def generar_random_datasets(num_vectores_r, num_vectores_q, dimensiones):
    dataset_r = numpy.random.rand(num_vectores_r, dimensiones).astype(numpy.float32)
    dataset_q = numpy.random.rand(num_vectores_q, dimensiones).astype(numpy.float32)
    print("R={} Q={}".format(dataset_r.shape, dataset_q.shape))
    return dataset_r, dataset_q


dataset_r, dataset_q = generar_random_datasets(100000, 1000, 128)
# para que los valores queden entre 0 y 100
dataset_q *= 100
dataset_r *= 100
print(dataset_r)

## Funciones para evaluar búsquedas

In [ ]:
import time


# calcular la calidad de la respuesta contra el resultado "ideal" dado por el linear scan
class Resultado:
    def __init__(self, nns_ideal, distancias_ideal, tiempo_ideal):
        self.nns_ideal = nns_ideal
        self.distancias_ideal = distancias_ideal
        self.tiempo_ideal = tiempo_ideal
        self.correctas = 0
        self.incorrectas = 0
        self.tiempo = 0
        self.precision = 0
        self.eficiencia = 0

    def evaluar_busqueda(self, nns, distancias, segundos):
        self.correctas = 0
        self.incorrectas = 0
        for i in range(len(self.nns_ideal)):
            if nns[i] == self.nns_ideal[i] or distancias[i] == self.distancias_ideal[i]:
                self.correctas += 1
            else:
                self.incorrectas += 1
        self.tiempo = segundos
        self.precision = self.correctas / (self.correctas + self.incorrectas)
        self.eficiencia = self.tiempo / self.tiempo_ideal

## Medir el tiempo que toma el Linear scan

In [ ]:
import pyflann
import time

# crear un objeto flann
flann = pyflann.FLANN()
flann.build_index(dataset_r, algorithm="linear")

t0 = time.time()
nns_LS, dists_LS = flann.nn_index(dataset_q, num_neighbors=1, cores=1)
segundos_LS = time.time() - t0
print("Linear scan toma {:.1f} segundos".format(segundos_LS))

# objeto para evaluar las busquedas posteriores
r = Resultado(nns_LS, dists_LS, segundos_LS)

## Construir un índice kdtree

In [ ]:
# construir el indice kdtree
t0 = time.time()
flann.build_index(dataset_r, algorithm="kdtree", trees=1)
segundos_construccion = time.time() - t0

print("construcción kd tree={:.2f} segundos".format(segundos_construccion))

## Búsqueda exacta usando el índice

En los datos aleatorios, la búsqueda exacta con un kd tree es mucho más lenta que linear scan.   

In [ ]:
# busqueda exacta del 1-NN usando el indice construido
t0 = time.time()
nn, dist = flann.nn_index(dataset_q, num_neighbors=1, cores=1, checks=-1)
segundos = time.time() - t0
r.evaluar_busqueda(nn, dist, segundos)

print(
    "kd tree EXACTO precision={:.1%} segundos={:.1f} ({:.1%})".format(
        r.precision, r.tiempo, r.eficiencia
    )
)

## Búsqueda aproximada con distintos niveles de aproximación

Notar que la calidad de las aproximaciones son muy malas (comparado con lo que se obtenía en los datasets reales).  
Esto es porque son datos aleatorios uniformes donde las distancias entre elementos son muy parecidas.

In [ ]:
for checks in (1, 10, 100, 500, 1000, 3000, 5000, 10000, 20000, 30000):
    t0 = time.time()
    nn, dist = flann.nn_index(dataset_q, num_neighbors=1, cores=1, checks=checks)
    segundos = time.time() - t0
    r.evaluar_busqueda(nn, dist, segundos)
    print(
        "kd tree checks={:5} precision={:5.1%} segundos={:4.2f} ({:6.1%})".format(
            checks, r.precision, r.tiempo, r.eficiencia
        )
    )

## Dimensión intrínseca del dataset aleatorio

Calcular un histograma con las distancias entre pares de elementos del dataset.   
Se calcula el valor de dimensión intrínseca que es igual o superior a la dimensionalidad del dataset.

In [ ]:
import random
import matplotlib.pyplot as plt


def muestra_de_distancias(dataset_r, cantidad):
    print(
        "calculando una muestra de {} distancias del conjunto {} vectores...".format(
            cantidad, dataset_r.shape
        )
    )
    distancias = list()
    num_vectors = dataset_r.shape[0]
    while len(distancias) < cantidad:
        pos1 = random.randint(0, num_vectors - 1)
        pos2 = random.randint(0, num_vectors - 1)
        if pos1 == pos2:
            continue
        vector1 = dataset_r[pos1]
        vector2 = dataset_r[pos2]
        distancia = numpy.linalg.norm(vector1 - vector2)
        distancias.append(distancia)
    print(
        "obtenidas {} distancias entre {} vectores (un {:.3%} del total de pares)".format(
            len(distancias),
            num_vectors,
            len(distancias) / (num_vectors * (num_vectors - 1)),
        )
    )
    return distancias


def histograma_distancias(dataset_r, cantidad):
    distancias = muestra_de_distancias(dataset_r, cantidad)
    plt.hist(distancias, bins=100)
    plt.xlabel("distancia")
    plt.ylabel("cantidad")
    plt.title("histograma de distancias")
    plt.show()
    media = numpy.average(distancias)
    varianza = numpy.var(distancias, ddof=1)
    print("distancia promedio entre pares = {:.3f}".format(media))
    print("varianza de distancias = {:.3f}".format(varianza))
    print(
        "dimension intrinseca del conjunto, rho = {:.1f}".format(
            (media * media) / (2 * varianza)
        )
    )


histograma_distancias(dataset_r, 1000000)

## Distancias entre datos reales (dataset_a)

Notar que la dimensión intrínseca de este dataset es mucho menor que su dimensionalidad, por lo que los índices lograrán buenos resultados (como se vio en semanas anteriores).


In [ ]:
datasetA_r = numpy.load("dataset_a_r.npy")
print("dataset A={}".format(datasetA_r.shape))

histograma_distancias(datasetA_r, 1000000)


## Comparar con datos reales (dataset_b)

La dimensión intrínseca de este dataset es menor que su dimensionalidad.

Notar que es incluso menor que la del dataset_a, pero en los ejemplos anteriores se veía que kdtree lograba mejores aproximaciones en dataset_a que en dataset_b.

In [ ]:
datasetB_r = numpy.load("dataset_b_r.npy")
print("dataset B={}".format(datasetB_r.shape))

histograma_distancias(datasetB_r, 1000000)

## Conclusión

Los datos aleatorios uniformes tienen una dimensión intrínseca muy alta, mucho mayor que la de los datos reales.

Una dimensión intrínseca alta significa que todos los vectores están a la misma distancia entre sí, por lo que el vecino más cercano se hace indistinguible del resto de vectores.

La búsqueda con índices no logra acelerar las búsquedas, porque hay muchos vectores a una distancia muy cercana a la del vecino cercano y deberá visitar prácticamente todos los nodos del árbol para asegurar que es el vecino más cercano.

Este comportamiento es llamado **"maldición de la dimensionalidad"** (que más bien podríamos llamar **"maldición de la uniformidad"**) que hace que los índices no funcionen bien en alta dimensión y que dificulta el análisis de datos con vectores de alta dimensión.